# Model to detect data exfiltration using clustering

## 0. Import Libraries

### 0.1. Standard Library

In [1]:
from dataclasses import (
    dataclass,
    replace,
)
import os
from pathlib import Path
from typing import (
    Literal,
)
import logging
import random
import shutil

### 0.2. External Libraries

In [2]:
from polars import selectors as cs
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim

## 1. Setup

### 1.1. Set Constants

#### 1.1.1. User Modifiable Constants

In [ ]:
# Name of the dataset file to be downloaded/used
DATASET_ROOT: Path = Path("data")
DATASET_PATH: Path = DATASET_ROOT / "dataset.csv"

# S3 Configuration 
USE_S3: bool = False 
S3_BUCKET_NAME: str = "logs-pobl"
S3_DATASET_KEY: str = "merged-logs/merged.csv"  
S3_MODELS_PREFIX: str = "production_models/" 
LOW_VARIANCE_THRESHOLD: float = 0.01 

PARAMETER_TUNING: bool = False

# Log file path
LOG_PATH: Path | None = (
    Path
        .cwd()
        .joinpath("classifier-test.log")
)

# Test against one-hot encoding.
PORT_PREPROCESSING_ONE_HOT: bool = True

# Correlation coefficient threshold for feature removal during preprocessing.
# Features with absolute correlation above this value (0.8) are considered highly correlated.
# During preprocessing, when two features exceed this threshold, one will be removed to reduce multicollinearity.
# Higher values (>0.8) are more conservative, removing only very strongly correlated features.
# Lower values (<0.8) are more aggressive, potentially removing more features.
# The value 0.8 is a common choice that balances information preservation with redundancy removal.
PREPROCESSING_CORRELATION_THRESHOLD: float = 0.8

# Correlation threshold for detecting potential data leakage during preprocessing.
# Features with absolute correlation coefficients >= 0.5 with the target variable 
# are considered suspicious and may represent leaked information.
# Using 0.5 is a conservative approach that can catch moderate correlations:
# - 0.8-1.0: Very strong correlation (definite leakage concern)
# - 0.6-0.8: Strong correlation (highly suspicious)
# - 0.5-0.6: Moderate correlation (potentially problematic)
# Consider adjusting based on domain knowledge and model performance.
PREPROCESSING_LEAKAGE_CORRELATION_THRESHOLD: float = 0.5

# Random seed for reproducibility (None for random behavior)
SEED: int | None = 42

# Whether to show output during processing
SHOW_OUTPUT: bool = True

#### 1.1.2. User Non-Modifiable Constants

In [4]:
DATASET_PATH: Path = Path("dataset.csv")

### 1.2. Setup Logging

In [5]:
logging.basicConfig(
    level=logging.INFO
)

## 2. Import Data

### 2.1. Get Dataset

To learn more about the dataset, we have visited the github page of the zeek package we download, where every feature is explained. The process of getting the dataset is documented in the memory, it's a merge of data gotten from Zeek and Zeek-flowmeter. For more information about the features we can visit https://github.com/zeek-flowmeter/zeek-flowmeter 

#### 2.1.1. Download Dataset

In [ ]:
##Logica de descargar de s3
import boto3
from botocore.exceptions import NoCredentialsError

def download_from_s3(bucket, s3_key, local_path):
    if not USE_S3:
        print("Skipping s3 download")
        return
    
    try:
        s3 = boto3.client('s3')
        print(f"Downloading {s3_key} from {bucket}")
        s3.download_file(bucket, s3_key, str(local_path))
        print(f"File saved at {local_path}")
    except NoCredentialsError:
        print("AWS not configured")
    except Exception as e:
        print(f"Error downloading from s3: {e}")

download_from_s3(S3_BUCKET_NAME, S3_DATASET_KEY, DATASET_PATH)

#### 2.1.2. Prepare Dataset

In [ ]:
# Esto abrirá un botoncito para que subas tu 'dataset.csv' desde tu PC
LAZYFRAME = pl.scan_csv(
    DATASET_PATH,
    null_values="NaN",
    infer_schema=True,
    infer_schema_length=None,
    rechunk=True,
    truncate_ragged_lines=True,
    separator=",",
)


In [8]:
# display(LAZYFRAME.head(5).collect())

# display(LAZYFRAME.describe())


## 3. Basic Preprocessing

Before we start using our data, we need to preprocess it 

### 3.1. Drop NULL Containing Rows

Handling missing values is a critical step in data preprocessing. Rows containing null values can introduce bias and cause runtime errors during model training. Since the dataset is sufficiently large, removing these incomplete records ensures data integrity without significantly reducing the statistical power of the sample

In [9]:
LAZYFRAME = LAZYFRAME.drop_nulls()

### 3.2. Handle Infinity Values

Infinite values (often caused by division by zero in flow feature calculations) are mathematically unusable for most machine learning algorithms because they disrupt distance calculations and gradient descents. We filter them out to ensure numerical stability.

In [11]:
LAZYFRAME = (
    LAZYFRAME
        .filter(
            pl.all_horizontal(
                cs.numeric()
                    .is_finite()
            )
        )
)

### 3.3. Handle Negative Protocol Header Lengths

The header length is a measure of the size of the headers, which cannot logically be negative or 0 in real-world scenarios. Negative or 0 values might indicate an issue, such as a bug in the software or incorrect packet parsing in the flow data being captured.

In [12]:
LAZYFRAME = (
    LAZYFRAME
        .filter(
            pl.all_horizontal(
                pl.col(r"^(Fwd|Bwd) Header Length$") > 0
            )
        )
)

### 3.4. Drop IP Addresses and Source Port

IP addresses (id.orig_h, id.resp_h) and ephemeral source ports (id.orig_p) are high-cardinality categorical identifiers. Including them can lead to the model learning specific "identities" rather than "behaviors" (attacks). For a generalizable intrusion detection system, we want the model to learn how the traffic looks (features), not who sent it.

In [ ]:
LAZYFRAME = (
    LAZYFRAME
        .drop([
            "id.orig_h",
            "id.resp_h",
            "id.orig_p",
            "id.resp_p",
        ])
)

### 3.5. Drop Identification Columns

The uid is a unique identifier for the connection log and carries no predictive information about the traffic's nature. Retaining it adds noise and dimensionality without adding value.

In [14]:
LAZYFRAME = (
    LAZYFRAME
        .drop(
            "uid", 
        )
)

### 3.6. Drop Timestamp Column

Clustering algorithms normally work with mathematical distances. They cannot calculate the distance between text strings such as “03/07/2017.” Furthermore, to detect intrusions, we are generally interested in the behavior of the flow (duration, bytes, packets) and not the exact time at which it occurred.

In [15]:
LAZYFRAME = (
    LAZYFRAME
        .drop(
            "ts_flow", 
        )
)

### 3.7. Drop Unnecessary Features

We drop the proto (protocol) feature because, in many network flow datasets, this feature is categorical (e.g., TCP, UDP) or highly invariant for specific attack types (e.g., all HTTPS attacks are TCP). For a numerical clustering approach, we focus on behavioral metrics (packet counts, durations) rather than static protocol identifiers. The same goes for the other features.

In [ ]:
LAZYFRAME = (
    LAZYFRAME
        .drop([
            "proto",
            "service",
            "history",
            "conn_state"
        ])
)

#### 3.8. Transform booleans into numbers

Some features like local_orig and local_resp are booleans, to not lose that data we transform them into booleans so we don't lose that information

In [ ]:
LAZYFRAME = LAZYFRAME.with_columns([
    pl.col("local_orig").cast(pl.Int8),
    pl.col("local_resp").cast(pl.Int8)
])

## 4. Exploratory Data Analysis (EDA)

In this section, we perform an exhaustive study of the variables and their distributions. We use visualization to understand data skewness, class balance, and feature correlations. These insights will justify our choice of scaling techniques (e.g., StandardScaler vs RobustScaler) and the suitability of clustering algorithms.

### 4.1. Data Overview

Get a high-level understanding of the dataset structure, dimensions, and data types after initial preprocessing.

Conclusion: The preprocessed dataset contains 371,621 network flow records with 70 features, representing a comprehensive set of network traffic characteristics. The feature set includes temporal metrics (flow durations, inter-arrival times), packet statistics (counts, sizes, payloads), TCP flag counters, and behavioral indicators (bulk transfers, window sizes, idle/active times). The mix of data types (Int64 and String) indicates that some features may require additional type conversion or encoding before clustering. This rich feature space provides strong potential for identifying patterns that distinguish normal traffic from data exfiltration attempts.

In [ ]:
df_eda = LAZYFRAME.collect()

print(f"\nDataset shape: {df_eda.shape[0]:,} rows × {df_eda.shape[1]} columns")
print(f"\nColumn names and types:")

for col in df_eda.columns:
    dtype = df_eda[col].dtype
    print(f"  {col:<40} {str(dtype):<15}")


### 4.2. Data Quality Check

We verify that our preprocessing successfully handled data quality issues (nulls, duplicates, infinities).

The dataset shows excellent quality after preprocessing: zero NULL values confirm complete data integrity, and the absence of infinite values in numeric columns ensures mathematical operations will be stable. However, the presence of 20,963 duplicate rows (approximately 5.6% of the dataset) suggests either repeated legitimate traffic patterns or potential data collection artifacts. These duplicates should be carefully considered while they could indicate normal recurring network behaviors, they might also introduce bias in clustering algorithms. Since in our case, we want to model the normalcy we leave the duplicated lines.

In [ ]:

null_counts = df_eda.null_count()
total_nulls = null_counts.sum_horizontal()[0]

print(f"\nTotal NULL values: {total_nulls}")

n_duplicates = df_eda.shape[0] - df_eda.unique().shape[0]
print(f"Duplicate rows: {n_duplicates}")

numeric_cols = df_eda.select(cs.numeric()).columns
has_inf = False

for col in numeric_cols:
    inf_count = df_eda.filter(~pl.col(col).is_finite()).shape[0]
    if inf_count > 0:
        print(f"'{col}' has {inf_count} infinite values")
        has_inf = True

if not has_inf:
    print(f"No infinite values found in numeric columns")

### 4.3. Statistical Summary

Understand the central tendency, spread, and range of each feature. This reveals the scale differences between features (critical for distance-based clustering) and helps identify potential outliers.

- Features with vastly different scales will dominate distance calculations
- Large differences between mean and median suggest skewness
- Large std relative to mean suggests high variability or outliers

The statistical summary reveals significant variation in network flow characteristics. Features exhibit extreme ranges across multiple orders of magnitude, particularly in temporal metrics (flow durations, inter-arrival times) and volume metrics (payload bytes, packet counts). This high variability necessitates careful feature scaling before clustering to prevent features with larger ranges from dominating distance calculations. The presence of features with very different scales (e.g., flag counts typically ranging 0-2 versus flow durations in microseconds) confirms that standardization or normalization will be critical for effective clustering performance.

In [ ]:
stats_df = df_eda.describe()
display(stats_df)

numeric_features = df_eda.select(cs.numeric()).columns
ranges = []

for col in numeric_features:
    col_data = df_eda[col]
    range_val = col_data.max() - col_data.min()
    ranges.append((col, range_val))

ranges_sorted = sorted(ranges, key=lambda x: x[1], reverse=True)

print("Features ranked by range (max - min):")
print("-"*80)
for i, (col, range_val) in enumerate(ranges_sorted[:10], 1):
    print(f"  {i:2d}. {col:<40} Range: {range_val:,.2f}")


### 4.4. Feature Distributions & Skewness

Examine the distribution shape of features to understand skewness and normality. Highly skewed distributions are common in network traffic data and may require log transformation or robust scaling methods for clustering algorithms.

- skewness < 0.5  : Fairly symmetric
- 0.5 < skewness < 1.0 : Moderately skewed
- skewness > 1.0  : Highly skewed


The top 15 most skewed features demonstrate severe right-skewed distributions (positive skewness), indicating that most network flows exhibit low values with occasional extreme outliers. This pattern is typical for network traffic where the majority of connections are brief and small, with rare cases of large data transfers.

The visualization of the top 6 most skewed features confirms these distributions are highly non-Gaussian, which has important implications for clustering algorithm selection - algorithms that assume Gaussian distributions (like some implementations of GMM) may struggle without transformation.

In [ ]:
numeric_features = df_eda.select(cs.numeric()).columns

skewness_data = []
for col in numeric_features:
    data = df_eda[col].to_numpy()
    mean = np.mean(data)
    std = np.std(data)
    if std > 0:
        skew = np.mean(((data - mean) / std) ** 3)
        skewness_data.append((col, skew))

skewness_sorted = sorted(skewness_data, key=lambda x: abs(x[1]), reverse=True)

print("\nTop 15 most skewed features:")
print("-"*80)

for i, (col, skew) in enumerate(skewness_sorted[:15], 1):
    direction = "right" if skew > 0 else "left"
    print(f"  {i:2d}. {col:<40} Skewness: {skew:>8.2f} ({direction})")


top_skewed = [col for col, _ in skewness_sorted[:6]]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_skewed):
    data = df_eda[feature].to_numpy()
    
    axes[i].hist(data, bins=50, color='steelblue', edgecolor='black', alpha=0.8)
    axes[i].set_title(f'{feature}\nSkewness: {skewness_sorted[i][1]:.2f}', fontsize=10)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_skewed):
    data = df_eda[feature].to_numpy()
    
    data_positive = data[data > 0]
    
    if len(data_positive) > 0:
        axes[i].hist(data_positive, bins=50, color='coral', edgecolor='black', alpha=0.8)
        axes[i].set_yscale('log')
        axes[i].set_title(f'{feature} (Log Scale)', fontsize=10)
        axes[i].set_xlabel('Value')
        axes[i].set_ylabel('Frequency (log)')
        axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 4.5. Outlier Detection & Analysis

We identify outliers using the Interquartile Range (IQR) method. In clustering, outliers can either represent anomalies of interest or noise that distorts cluster formation. We analyze their prevalence to decide on handling strategies.

We can see that there are many variables which have high number of outliers

In [ ]:
outlier_stats = []

for col in numeric_features:
    data = df_eda[col].to_numpy()
    
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (data < lower_bound) | (data > upper_bound)
    n_outliers = np.sum(outliers)
    pct_outliers = (n_outliers / len(data)) * 100
    
    outlier_stats.append((col, n_outliers, pct_outliers))

outlier_stats_sorted = sorted(outlier_stats, key=lambda x: x[2], reverse=True)

print("\nFeatures with highest outlier percentage:")
for i, (col, n_out, pct_out) in enumerate(outlier_stats_sorted[:15], 1):
    print(f"  {i:2d}. {col:<40} {n_out:>6} ({pct_out:>5.2f}%)")
print("\n" + "="*80)

top_outlier_features = [col for col, _, _ in outlier_stats_sorted[:6]]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_outlier_features):
    data = df_eda[feature].to_numpy()
    
    axes[i].boxplot(data, vert=True, patch_artist=True,
                    boxprops=dict(facecolor='lightblue'),
                    medianprops=dict(color='red', linewidth=2))
    axes[i].set_title(f'{feature}', fontsize=10)
    axes[i].set_ylabel('Value')
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 4.6. Correlation Analysis

Examine linear relationships between features. High correlations indicate redundancy, which can bias clustering algorithms by giving disproportionate weight to correlated feature groups. We assume that more than 0.8 is highly correlated.

The correlation analysis identified 39 pairs of features with strong correlations (|r| ≥ 0.8), revealing significant multicollinearity in the dataset. The top correlations show that many features are mathematically or logically related. This means that there could be some redundant information, which increases computational cost without adding useful information

In [ ]:
corr_matrix_df = df_eda.select(numeric_features).corr()

plt.figure(figsize=(20, 16))
plt.imshow(corr_matrix_df, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label='Correlation Coefficient')

plt.xticks(range(len(numeric_features)), numeric_features, rotation=90, fontsize=8)
plt.yticks(range(len(numeric_features)), numeric_features, fontsize=8)

plt.title('Feature Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

strong_correlations = []
for i in range(len(numeric_features)):
    for j in range(i+1, len(numeric_features)):
        corr_val = corr_matrix_df[i, j]
        if abs(corr_val) >= 0.8: 
            strong_correlations.append((numeric_features[i], numeric_features[j], corr_val))

strong_correlations_sorted = sorted(strong_correlations, key=lambda x: abs(x[2]), reverse=True)


print("STRONG CORRELATIONS (|r| >= 0.8)")

print(f"\nTotal pairs with strong correlation: {len(strong_correlations_sorted)}")
print("\nTop 20 strongest correlations:")
print("-"*80)

for i, (feat1, feat2, corr) in enumerate(strong_correlations_sorted[:20], 1):
    print(f"  {i:2d}. {feat1:<35} <-> {feat2:<35} r={corr:>7.4f}")

print("\n" + "="*80)

### 4.7 Feature Variance Analysis

Identify low-variance features that provide little information for distinguishing between samples. Features with near-zero variance are candidates for removal as they don't contribute to cluster separation.

The variance analysis reveals a striking dichotomy between low and high variance features:

Low-variance features (bottom 15):

Several features show near-zero variance (bwd_URG_flag_count, fwd_URG_flag_count5, bwd_subflow_pkts with Var ≈ 0). These constant or near-constant features provide minimal discriminative power for clustering. Should be considered for removal as they won't help separate different traffic patterns

There is also a high-variance features which show extreme variability and could greatly influence distance calculations. These might need to be standarized

In [ ]:
variance_data = []
for col in numeric_features:
    var = df_eda[col].var()
    std = df_eda[col].std()
    variance_data.append((col, var, std))

variance_sorted = sorted(variance_data, key=lambda x: x[1])

print("\nFeatures with lowest variance (top 15):")
print("-"*80)
for i, (col, var, std) in enumerate(variance_sorted[:15], 1):
    print(f"  {i:2d}. {col:<40} Var: {var:>12.4f}  Std: {std:>12.4f}")

print("\nFeatures with highest variance (top 15):")
print("-"*80)
for i, (col, var, std) in enumerate(reversed(variance_sorted[-15:]), 1):
    print(f"  {i:2d}. {col:<40} Var: {var:>12.2f}  Std: {std:>12.2f}")

variances = [var for _, var, _ in variance_data]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(variances, bins=30, color='teal', edgecolor='black', alpha=0.8)
plt.xlabel('Variance')
plt.ylabel('Number of Features')
plt.title('Distribution of Feature Variances')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(np.log10(np.array(variances) + 1), bins=30, color='orange', edgecolor='black', alpha=0.8)
plt.xlabel('Log10(Variance + 1)')
plt.ylabel('Number of Features')
plt.title('Distribution of Feature Variances (Log Scale)')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Advanced Preprocessing

Now that we have done the EDA, we can make informed transformations on our data.

#### INSIGHT 1: Extreme Skewness (Section 4.4)
**Finding**: Many features with skewness > 5.0  
**Decision**: Use RobustScaler (not StandardScaler)  
**Reason**: StandardScaler assumes Gaussian distribution (violated by extreme skewness)

#### INSIGHT 2: High Outlier Prevalence (Section 4.5)  
**Finding**: 40% of features have >15% outliers  
**Decision**: PRESERVE outliers (do not remove)  
**Reason**: In intrusion detection, outliers ARE the anomalies we want to detect

#### INSIGHT 3: Significant Multicollinearity (Section 4.6)
**Finding**: Lots of pairs of features with |r| ≥ 0.8  
**Decision**: Remove redundant features  
**Strategy**: For each correlated pair, drop the feature with higher average correlation

#### INSIGHT 4: Vastly Different Feature Scales (Section 4.3)
**Finding**: Ranges from 10⁰ to 10⁶ between features  
**Decision**: Feature scaling is CRITICAL  
**Choice**: RobustScaler (confirmed by Insights 1 and 2)

#### INSIGHT 5: Features with very low variance (Section 4.7)
**Finding**: Some features have a really low variance  
**Decision**: Remove near constant features  
**Choice**: We drop the features below the specificed threshold, 0.01  

### 5.1. Drop Highly Correlated Features

Based on the correlation matrix we saw above, we identified pairs of features that carry almost identical information (Multicollinearity).

To resolve this, we apply an automated selection strategy:

For each correlated pair (correlation > 0.8), we compare the Mean Absolute Correlation of each feature against the rest of the dataset.

We drop the feature that has the higher average correlation with all other features.

In [ ]:
corr_matrix_np = corr_matrix_df.to_numpy()

# 1. Calculate Mean Absolute Correlation for each feature
# This helps us decide WHICH of the two correlated features to kill (the most redundant one)
mean_correlations = {}

for i, feat in enumerate(numeric_features):
    # Calculate mean of absolute correlations (excluding self-correlation at index i)
    # We iterate over the row 'i' of the matrix
    corrs = [abs(corr_matrix_np[i, j]) for j in range(len(numeric_features)) if i != j]
    mean_correlations[feat] = sum(corrs) / len(corrs) if corrs else 0

# 2. Decide which columns to drop
cols_to_drop = set()

# strong_correlations format comes from your previous cell: [(feat1, feat2, corr_value), ...]
for feat1, feat2, _ in strong_correlations:
    # Skip if one is already marked for death
    if feat1 in cols_to_drop or feat2 in cols_to_drop:
        continue
        
    # Drop the feature that is "more correlated" with everything else (more generic/redundant)
    if mean_correlations[feat1] > mean_correlations[feat2]:
        cols_to_drop.add(feat1)
    else:
        cols_to_drop.add(feat2)

# 3. Apply changes to the LAZYFRAME
print(f"Dropping {len(cols_to_drop)} columns due to high correlation (> 0.8):")
for col in sorted(cols_to_drop):
    print(f"\t- {col}")

LAZYFRAME = LAZYFRAME.drop(list(cols_to_drop))

df_eda = LAZYFRAME.collect()
numeric_features = [col for col in numeric_features if col not in cols_to_drop]

print(f"\nRemaining features: {len(numeric_features)}")

### 5.2. Drop Low-Variance Features


We saw earlier that there are many features that barely change, we don't want to keep these since they will only increase computational cost without actually helping us take decisions. We delete the ones that are below the defined threshold LOW_VARIANCE_THRESHOLD

In [ ]:
# Calculate variance for all numeric features
variance_data = []
for col in numeric_features:
    var = df_eda[col].var()
    variance_data.append((col, var))

# Identify features with variance below threshold
low_variance_features = [col for col, var in variance_data if var < LOW_VARIANCE_THRESHOLD]

print(f"Dropping {len(low_variance_features)} low-variance features (variance < {LOW_VARIANCE_THRESHOLD}):")
for col in low_variance_features:
    var = next(v for c, v in variance_data if c == col)
    print(f"\t- {col} (variance: {var:.6f})")

# Drop from LazyFrame
if low_variance_features:
    LAZYFRAME = LAZYFRAME.drop(low_variance_features)
    df_eda = LAZYFRAME.collect()
    numeric_features = [f for f in numeric_features if f not in low_variance_features]
    
    print(f"\nRemaining features after variance filtering: {len(numeric_features)}")
else:
    print("No features below variance threshold found.")

### 5.3. Feature Scaling

Since we saw that the features are scaled very differently, we are using RobustScaler to deal with the different scales that the features have. We use RobustScaler instead of StandardScaler since it deals better with outliers, as we could see during the 4.5, there were many outliers in lots of the features of the dataset. We also saw high skewness, which doesn't work well with StandardScaler.

Clustering algorithms like K-Means rely on distance metrics (Euclidean distance) to form clusters. In our dataset, features vary wildly in magnitude:
 * `flow_duration`: varies from 0 to 60+ seconds.
 * `fwd_pkts_payload.tot`: varies from 0 to millions of bytes.

Now we can see features that previously ranged in the millions (e.g., Bytes) and features ranging from 0 to 1 (e.g., Ratios) are now mapped to a comparable range (mostly centered around 0 due to median centering).

Outlier Preservation: Crucially, the extreme values (likely attacks) have not been squashed into a 0-1 range (as MinMaxScaler would do). Instead, they remain as distinct high-value points (e.g., > 10 or > 20 standard deviations), allowing the clustering algorithm to detect them as anomalies rather than merging them with normal traffic.

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler
import pandas as pd

# 1. Prepare the data
print(f"Preparing to scale {len(numeric_features)} features")

# We convert the Polars DataFrame to Pandas for Scikit-Learn compatibility
X_raw = df_eda.select(numeric_features).to_pandas()

# 2. Apply RobustScaler
# We use RobustScaler to handle the heavy tails/outliers typical in network data
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_raw)

# 3. Verification
# We convert back to a DataFrame just to visualize the new ranges.
df_scaled_check = pd.DataFrame(X_scaled, columns=numeric_features)

print(f"\nData successfully scaled using RobustScaler.")
print(f"Shape: {X_scaled.shape}")
print("\nNew Feature Ranges (Check for scaling consistency):")
display(df_scaled_check.describe().loc[['min', '50%', 'max', 'std']].transpose().head(10))


## 6. Post-Scaling Analysis

### 6.1. Dimensionality Reduction Exploration

We use PCA, t-SNE, and UMAP to project high-dimensional data into 2D/3D for visualization. This helps us assess whether natural clusters exist in the data and how much information can be retained with fewer dimensions.

High Dimensionality & Complexity: The PCA analysis reveals that the dataset is complex. To explain just 80% of the variance, we need 14 Principal Components (out of 50 original features). This indicates that the information is spread across many variables and cannot be easily compressed into a few linear dimensions without significant loss.

Limitations of Linear Projection: The 2D PCA projection only captures 32.32% of the total variance. This low percentage confirms that a simple linear 2D plot is insufficient to visualize the true structure of the data.

Manifold Structure: While PCA struggles to separate the groups in 2D, the application of non-linear techniques like t-SNE or UMAP (if visualized) typically reveals much clearer local structures and clusters.

We cannot rely on a 2D or 3D reduced dataset for the final clustering model if we want high accuracy. The clustering algorithm should be applied to the scaled high-dimensional feature space (or a PCA-reduced space with at least 14 components) to capture the full behavioral patterns of the attacks.

Although the results, are useful to make an idea, we need to properly scale the data to come to a better conclusion

#### Cambiar a datos reales

#### 6.1.1. PCA Analysis

In [ ]:
from sklearn.decomposition import PCA


print("DIMENSIONALITY REDUCTION EXPLORATION")
print(f"\nOriginal dimensions: {X_scaled.shape[1]} features")
print("\nStandardization applied: mean=0, std=1 for all features")


print("PCA (Principal Component Analysis)")

pca_full = PCA()
pca_full.fit(X_scaled)

cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1

print(f"\nComponents needed to explain:")
print(f"  - 80% variance: {n_components_80} components")
print(f"  - 90% variance: {n_components_90} components")
print(f"  - 95% variance: {n_components_95} components")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Individual variance per component
axes[0].bar(range(1, min(21, len(pca_full.explained_variance_ratio_) + 1)), 
            pca_full.explained_variance_ratio_[:20], 
            color='steelblue', alpha=0.8)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Variance Explained by Each Component (Top 20)')
axes[0].grid(axis='y', alpha=0.3)

# Cumulative variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
             marker='o', markersize=4, color='coral', linewidth=2)
axes[1].axhline(y=0.80, color='green', linestyle='--', label='80%')
axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90%')
axes[1].axhline(y=0.95, color='red', linestyle='--', label='95%')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# PCA 2D projection
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

print(f"\n2D PCA captures {pca_2d.explained_variance_ratio_.sum():.2%} of total variance")

#### 6.1.2. 2D Visualizations

- Distinct clusters suggest the data has natural groupings
- Overlapping points suggest harder clustering problem
- t-SNE/UMAP better preserve local structure than PCA

In [ ]:
# Sample data if too large for t-SNE/UMAP
from sklearn.manifold import TSNE


sample_size = min(5000, len(X_scaled))
if len(X_scaled) > sample_size:
    print(f"\nSampling {sample_size} points for t-SNE and UMAP (computational efficiency)")
    np.random.seed(SEED)
    sample_idx = np.random.choice(len(X_scaled), sample_size, replace=False)
    X_sampled = X_scaled[sample_idx]
else:
    X_sampled = X_scaled
    sample_idx = np.arange(len(X_scaled))

# t-SNE
print("\nComputing t-SNE projection")
tsne = TSNE(n_components=2, random_state=SEED, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_sampled)

# UMAP
print("Computing UMAP projection")
try:
    from umap import UMAP
    umap_model = UMAP(n_components=2, random_state=SEED, n_neighbors=15, min_dist=0.1)
    X_umap = umap_model.fit_transform(X_sampled)
    has_umap = True
except ImportError:
    has_umap = False

# Plot all projections
n_plots = 3 if has_umap else 2
fig, axes = plt.subplots(1, n_plots, figsize=(6*n_plots, 5))

if n_plots == 2:
    axes = [axes[0], axes[1]]

# PCA plot
axes[0].scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], alpha=0.5, s=10, c='steelblue')
axes[0].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
axes[0].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
axes[0].set_title('PCA Projection')
axes[0].grid(alpha=0.3)

# t-SNE plot
axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], alpha=0.5, s=10, c='coral')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
axes[1].set_title('t-SNE Projection')
axes[1].grid(alpha=0.3)

# UMAP plot
if has_umap:
    axes[2].scatter(X_umap[:, 0], X_umap[:, 1], alpha=0.5, s=10, c='mediumseagreen')
    axes[2].set_xlabel('UMAP 1')
    axes[2].set_ylabel('UMAP 2')
    axes[2].set_title('UMAP Projection')
    axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()



### 6.2. Distance & Density Analysis

Analyze the distribution of pairwise distances between samples. Clustering algorithms like K-Means rely on distance metrics, so understanding distance distributions helps assess data structure and potential cluster separability.

- Multimodal distance distribution may indicate multiple clusters
- Uniform distribution suggests data is evenly spread (harder to cluster)


The mean distance is approximately 6.25, providing a baseline for what constitutes "near" or "far" in this high-dimensional space.

Since the distribution is unimodal but wide, it suggests a continuous variation in traffic behavior, which might require density-based algorithms (like DBSCAN) or careful tuning of $k$ in K-Means.

#### CAMBIAR CON EL VALOR REAL!!!

In [ ]:
# Sample for computational efficiency
sample_for_distance = min(2000, len(X_scaled))
if len(X_scaled) > sample_for_distance:
    np.random.seed(SEED)
    dist_sample_idx = np.random.choice(len(X_scaled), sample_for_distance, replace=False)
    X_dist_sample = X_scaled[dist_sample_idx]
else:
    X_dist_sample = X_scaled

print(f"\nCalculating pairwise distances for {len(X_dist_sample)} samples...")

# Calculate pairwise Euclidean distances
from scipy.spatial.distance import pdist, squareform

distances = pdist(X_dist_sample, metric='euclidean')

print(f"\nDistance statistics:")
print(f"  - Mean distance: {np.mean(distances):.4f}")
print(f"  - Median distance: {np.median(distances):.4f}")
print(f"  - Std distance: {np.std(distances):.4f}")
print(f"  - Min distance: {np.min(distances):.4f}")
print(f"  - Max distance: {np.max(distances):.4f}")

# Plot distance distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(distances, bins=50, color='purple', edgecolor='black', alpha=0.8)
plt.xlabel('Euclidean Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Pairwise Distances')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(distances, bins=50, color='purple', edgecolor='black', alpha=0.8, cumulative=True, density=True)
plt.xlabel('Euclidean Distance')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative Distribution of Distances')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()



### 6.3. Hopkins Statistic (Cluster Tendency)

The Hopkins statistic tests whether the data has a clustering tendency or is uniformly distributed. Values close to 0.5 indicate uniform distribution (no clusters), while values significantly above 0.5 suggest clustered structure. Values higher than 0.8 indicate high clustering tendency.

We can see that the hopkins Statistic is 0.9779.
Since H ≥ 0.8: Strong clustering tendency - data is highly clusterable, which is great since it means it can be clusterized easily.
#### CAMBIAR CON EL VALOR REAL!!!

In [ ]:
def hopkins_statistic(X, sample_size=None):
    if sample_size is None:
        sample_size = min(int(len(X) * 0.1), 500)
    
    n = len(X)
    d = X.shape[1]
    
    # Sample random points from dataset
    np.random.seed(SEED)
    sample_indices = np.random.choice(n, sample_size, replace=False)
    X_sample = X[sample_indices]
    
    # Generate uniform random points in the same space
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)
    X_random = np.random.uniform(X_min, X_max, size=(sample_size, d))
    
    # Calculate distances to nearest neighbors
    from sklearn.neighbors import NearestNeighbors
    
    # Distances from random points to real data
    nbrs_real = NearestNeighbors(n_neighbors=1).fit(X)
    u_distances, _ = nbrs_real.kneighbors(X_random)
    u = u_distances.sum()
    
    # Distances from sample points to rest of real data
    nbrs_sample = NearestNeighbors(n_neighbors=2).fit(X)  # n_neighbors=2 to exclude self
    w_distances, _ = nbrs_sample.kneighbors(X_sample)
    w = w_distances[:, 1].sum()  # Take second nearest (first is self)
    
    # Hopkins statistic
    H = u / (u + w)
    
    return H

# Sample for efficiency
hopkins_sample_size = min(1000, len(X_scaled))
if len(X_scaled) > hopkins_sample_size:
    np.random.seed(SEED)
    hopkins_idx = np.random.choice(len(X_scaled), hopkins_sample_size, replace=False)
    X_hopkins = X_scaled[hopkins_idx]
else:
    X_hopkins = X_scaled

print(f"\nCalculating Hopkins statistic using {len(X_hopkins)} samples...")
H = hopkins_statistic(X_hopkins)

print(f"\nHopkins Statistic: {H:.4f}")
print("\nInterpretation:")
if H < 0.3:
    print("  → H < 0.3: Data is regularly spaced (unlikely to have clusters)")
elif H < 0.5:
    print("  → 0.3 ≤ H < 0.5: Data tends toward uniform distribution")
elif H < 0.8:
    print("  → 0.5 ≤ H < 0.8: Moderate clustering tendency")
else:
    print("  → H ≥ 0.8: Strong clustering tendency - data is highly clusterable")


## 7. Modeling & Validation

In this section, we proceed start building the machine learning models to detect potential data exfiltration. Since we are working with an unlabeled dataset, we apply Unsupervised Learning techniques to discover natural structures and patterns within the high-dimensional feature space. First of all, we will try to find what the optimal number of clusters is

### 7.1. Optimal Number of Clusters - Elbow Method

The elbow method calculates within-cluster sum of squares (inertia) for different values of k. The "elbow" point indicates where adding more clusters yields diminishing returns, suggesting an optimal k.

The plot shows a sharp decrease in inertia as $k$ increases from 1 to 3. The curve begins to flatten out significantly around $k=3$ or $k=4$, indicating the "elbow" point. This suggests that adding more clusters beyond this point yields diminishing returns in terms of variance explanation.

We can assume that one cluster is the normal traffic, a second one could be the exfiltrations we want to find and the third would be other anomalous activity

#### CAMBIAR CON EL VALOR REAL!!!

In [ ]:
from sklearn.cluster import KMeans


# Use PCA-reduced data for faster computation
pca_95 = PCA(n_components=0.95, random_state=SEED)
X_pca_reduced = pca_95.fit_transform(X_scaled)
print(f"Reduced to {X_pca_reduced.shape[1]} components")

max_samples_clustering = 10000
if len(X_pca_reduced) > max_samples_clustering:
    print(f"Sampling {max_samples_clustering} points for clustering analysis...")
    np.random.seed(SEED)
    cluster_sample_idx = np.random.choice(len(X_pca_reduced), max_samples_clustering, replace=False)
    X_cluster = X_pca_reduced[cluster_sample_idx]
else:
    X_cluster = X_pca_reduced

print(f"\nAnalyzing k from 2 to 15")


k_range = range(2, 16)
inertias = []
silhouette_scores = []

for k in k_range:
    print(f"  Computing k={k}...", end=" ")
    
    # Fit KMeans
    kmeans = KMeans(n_clusters=k, random_state=SEED, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(X_cluster)
    
    # Calculate inertia
    inertias.append(kmeans.inertia_)
    
    # Calculate silhouette score
    from sklearn.metrics import silhouette_score
    sil_score = silhouette_score(X_cluster, labels, sample_size=min(5000, len(X_cluster)))
    silhouette_scores.append(sil_score)
    
    print(f"Inertia: {kmeans.inertia_:.2f}, Silhouette: {sil_score:.4f}")

print("\n" + "-"*80)

#### 7.1.1. Elbow Plot


- Elbow: Look for the 'bend' where inertia decrease slows
- Silhouette: Higher is better (range: -1 to 1)
  * '> 0.5: Strong cluster structure
  * 0.25-0.5: Moderate structure
  * < 0.25: Weak or artificial structure

As expected, inertia decreases as $k$ increases, since more centroids allow for closer proximity to data points. However, we look for the "inflection point" where the rate of decrease shifts from rapid to marginal.

The plot reveals a distinct elbow at $k=3$ (or $k=4$, look at your graph). Before this point, adding a cluster significantly reduces variance (compaction). After this point, the curve flattens, indicating diminishing returns.

#### CAMBIAR CON EL VALOR REAL!!!

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Elbow plot
axes[0].plot(k_range, inertias, marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Within-Cluster Sum of Squares (Inertia)', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14)
axes[0].grid(alpha=0.3)
axes[0].set_xticks(k_range)

# Silhouette plot
axes[1].plot(k_range, silhouette_scores, marker='o', linewidth=2, markersize=8, color='coral')
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score vs k', fontsize=14)
axes[1].grid(alpha=0.3)
axes[1].set_xticks(k_range)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Find best k by silhouette score
best_k_silhouette = k_range[np.argmax(silhouette_scores)]
print(f"\nBest k by Silhouette Score: {best_k_silhouette} (score: {max(silhouette_scores):.4f})")



#### 7.1.2. Gap Statistic

The Gap statistic compares the within-cluster dispersion to that expected under a null reference distribution (uniform random data). A large gap indicates that the clustering structure is significantly better than random.

The "Gap" ($G_k$) measures how much better our clustering is compared to random noise. We seek the $k$ that maximizes this gap.

The Gap Statistic reaches its maximum (or first significant peak) at $k=3$. 
#### CAMBIAR CON EL VALOR REAL!!!

In [ ]:
def gap_statistic(X, k_range, n_refs=5, random_state=None):
    gaps = []
    s_k = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
        kmeans.fit(X)
        W_k = kmeans.inertia_
        
        W_kb_refs = []
        for _ in range(n_refs):
            X_ref = np.random.uniform(X.min(), X.max(), size=X.shape)
            
            kmeans_ref = KMeans(n_clusters=k, random_state=random_state, n_init=10)
            kmeans_ref.fit(X_ref)
            W_kb_refs.append(kmeans_ref.inertia_)
        
        gap = np.log(np.mean(W_kb_refs)) - np.log(W_k)
        gaps.append(gap)
        
        sdk = np.std(np.log(W_kb_refs))
        s_k.append(sdk * np.sqrt(1 + 1/n_refs))
    
    return np.array(gaps), np.array(s_k)

n_refs = 5
print(f"\nCalculating Gap statistic (using {n_refs} reference datasets per k)...")
gaps, s_k = gap_statistic(X_cluster, k_range, n_refs=n_refs, random_state=SEED)

# Find optimal k: first k where Gap(k) >= Gap(k+1) - s(k+1)
k_optimal_gap = None
for i in range(len(k_range) - 1):
    if gaps[i] >= gaps[i+1] - s_k[i+1]:
        k_optimal_gap = k_range[i]
        break

if k_optimal_gap is None:
    k_optimal_gap = k_range[np.argmax(gaps)]

print(f"Optimal k by Gap Statistic: {k_optimal_gap}")

# Plot gap statistic
plt.figure(figsize=(10, 6))
plt.errorbar(k_range, gaps, yerr=s_k, marker='o', linewidth=2, markersize=8, 
             color='mediumseagreen', ecolor='gray', capsize=5, capthick=2)
plt.axvline(x=k_optimal_gap, color='red', linestyle='--', linewidth=2, label=f'Optimal k={k_optimal_gap}')
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Gap Statistic', fontsize=12)
plt.title('Gap Statistic Method', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.xticks(k_range)
plt.tight_layout()
plt.show()

print("\n" + "="*80)

### 7.2. Clustering Algorithm Comparison

This part compares three clustering methods (K-Means, Ward, and HDBSCAN) on the same dataset. We chose these three algorithms because they represent complementary clustering paradigms, centroid-based (K-Means), hierarchical (Ward), and density-based (HDBSCAN), so we can compare different assumptions about cluster shape, scale and noise. Other approaches (e.g., LOF, Isolation Forest, or autoencoders) were omitted here because they focus on pointwise outlier scoring or require very different preprocessing and complexity, which would make a direct comparison harder.

The data is high-dimensional (33 features) but first projected to 2D through PCA, which preserves most of the variance like we saw earlier (≈96%).
HDBSCAN uses density estimation in the reduced space to discover natural groups without assuming a fixed number of clusters.
Ward and K-Means both assume a predefined “k=2” grouping, which like saw earlier, seems like the best option
The script also evaluates clusters with Davies-Bouldin and Calinski-Harabasz metrics, and visualizes the results in PCA space.

For detecting data exfiltration, HDBSCAN is the better choice. K-Means produces two large, well-separated clusters but treats all samples as normal, which hides rare and suspicious behaviors. HDBSCAN, on the other hand, isolates a small number of dense, stable patterns and classifies most irregular samples as noise. That behavior is desirable for security monitoring because the “noise” region naturally captures rare, potentially malicious outliers that could correspond to exfiltration events. Ward underperforms and discards structure, so it's not very promising. Seeing these results, HDBSCAN seems like the best option

In [ ]:
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import hdbscan

k_to_test = 2
X_train = X_scaled
sample_size = 2500

print(f"\nTesting clustering")
print(f"Training on: {X_train.shape[0]} samples / {X_train.shape[1]} features")
print(f"Visualization: PCA 2D\n")

pca = PCA(n_components=2, random_state=SEED)
X_pca_2d = pca.fit_transform(X_train)

if X_train.shape[0] > sample_size:
    idx = np.random.choice(len(X_train), sample_size, replace=False)
    X_sample = X_train[idx]
    X_sample_pca = X_pca_2d[idx]
else:
    X_sample = X_train
    X_sample_pca = X_pca_2d

algorithms = {
    'K-Means': KMeans(n_clusters=k_to_test, random_state=SEED, n_init=10),
    'HDBSCAN': hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=10, metric='euclidean'),
    'Ward (sample)': AgglomerativeClustering(n_clusters=k_to_test, linkage='ward'),
}

results = {}

for name, algo in algorithms.items():
    print(f"Fitting {name}...")

    if name == "Ward (sample)":
        labels_sample = algo.fit_predict(X_sample)
        full_labels = np.zeros(len(X_train)) - 1
        if len(X_train) > sample_size:
            full_labels[idx] = labels_sample
        else:
            full_labels = labels_sample
        labels = full_labels

    elif name == "HDBSCAN":
        labels = algo.fit_predict(X_pca_2d)

        unique, counts = np.unique(labels[labels >= 0], return_counts=True)
        if len(unique) > 3:
            top = unique[np.argsort(-counts)[:3]]
            new_labels = np.zeros_like(labels) - 1
            for new_k, old_k in enumerate(top):
                new_labels[labels == old_k] = new_k
            labels = new_labels

    else:
        labels = algo.fit_predict(X_train)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)

    if n_clusters > 1:
        mask = labels != -1
        if mask.sum() > 0:
            db_score = davies_bouldin_score(X_train[mask], labels[mask])
            ch_score = calinski_harabasz_score(X_train[mask], labels[mask])
        else:
            db_score = ch_score = np.nan
    else:
        db_score = ch_score = np.nan

    results[name] = {
        'labels': labels,
        'n_clusters': n_clusters,
        'n_noise': n_noise,
        'davies_bouldin': db_score,
        'calinski_harabasz': ch_score,
    }

    print(f"  -> Clusters: {n_clusters} | Noise: {n_noise}")
    if not np.isnan(db_score):
        print(f"  -> Davies-Bouldin: {db_score:.4f}")
        print(f"  -> Calinski-Harabasz: {ch_score:.2f}")
    print("-" * 40)

fig, axes = plt.subplots(1, len(results), figsize=(20, 6))

for i, (name, result) in enumerate(results.items()):
    labels = result['labels']
    axes[i].scatter(X_pca_2d[:,0], X_pca_2d[:,1],
                    c=labels, cmap='tab10', alpha=0.6, s=10)

    if result['n_noise'] > 0:
        mask = labels == -1
        axes[i].scatter(X_pca_2d[mask,0], X_pca_2d[mask,1],
                        c='black', marker='x', s=20, label='Noise')

    title = f"{name}\n{result['n_clusters']} clusters"
    if not np.isnan(result['davies_bouldin']):
        title += f"\nDB: {result['davies_bouldin']:.3f}"

    axes[i].set_title(title)
    axes[i].set_xlabel("PC1")
    axes[i].set_ylabel("PC2")
    axes[i].grid(alpha=0.3)
    if result['n_noise'] > 0:
        axes[i].legend()

plt.tight_layout()
plt.show()


### 7.3. HDBSCAN Hyperparameter Tunin

Before we can use our model, we have to tune the hyperparameters so they are as effective as they can be. For that we do the hyperparameter tuning using a grid. Since this project has computational and time limitations, we consider this the best approach, were we fine tune only the most important hyperparameters with only some logical values.

1. min_cluster_size (500-3000):
   - Minimum points to form a cluster
   - Lower values (500-1000): Detect smaller exfiltration campaigns
   - Higher values (2000-3000): Focus on large-scale attack patterns
   - Critical for distinguishing isolated exfiltration from bulk attacks

2. min_samples (5-20):
   - Controls noise sensitivity and cluster density requirements
   - Lower values (5-10): More aggressive clustering, catches subtle exfiltration
   - Higher values (15-20): Conservative clustering, reduces false positives
   - Balances between detecting stealthy exfiltration vs. noisy baseline traffic

3. metric (euclidean, manhattan, cosine, chebyshev):
   - Distance calculation method
   - Euclidean: Standard for continuous features, penalizes outliers
   - Manhattan: More robust to outliers, better for mixed feature scales
   - Cosine distance: Captures directional similarity in flow patterns. Exfiltration often has characteristic ratios (bytes_sent/bytes_received, packet timing) that cosine distance identifies regardless of absolute volume.
   - Chebyshev distance: Maximum difference across any dimension. Useful for detecting exfiltration that differs from normal traffic in just one extreme feature (e.g., unusually long connection duration or huge packets).

4. cluster_selection_epsilon (0.0-0.3):
   - Merges clusters closer than epsilon distance
   - 0.0: No merging, maximum granularity
   - 0.1-0.3: Consolidates similar attack types/exfiltration methods
   - Prevents over-segmentation of related malicious activities

Each configuration is evaluated using the DBCV (Density-Based Cluster Validation) score, HDBSCAN's native validation metric that measures cluster validity without requiring ground truth labels, alongside traditional metrics like Davies-Bouldin and Calinski-Harabasz scores. Configurations are filtered by practicability.

In [ ]:
from itertools import product
import hdbscan
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score

# Define hyperparameter grid
if PARAMETER_TUNING:
    param_grid = {
        'min_cluster_size': [500, 1000, 2000, 3000],
        'min_samples': [5, 10, 15, 20],
        'metric': ['euclidean', 'manhattan', 'cosine', 'chebyshev'],
        'cluster_selection_epsilon': [0.0, 0.1, 0.2, 0.3]
    }

    # For computational efficiency, sample 
    max_tuning_samples = 20000
    if len(X_scaled) > max_tuning_samples:
        print(f"\nSampling {max_tuning_samples} points for hyperparameter tuning")
        np.random.seed(SEED)
        tune_idx = np.random.choice(len(X_scaled), max_tuning_samples, replace=False)
        X_tune = X_scaled[tune_idx]
    else:
        X_tune = X_scaled
        
    print(f"Tuning on: {X_tune.shape[0]} samples")
    print(f"\nGrid size: {len(list(product(*param_grid.values())))} combinations")
    print("This may take several minutes...\n")

    # Store results
    tuning_results = []

    # Grid search
    for min_cs in param_grid['min_cluster_size']:
        for min_s in param_grid['min_samples']:
            for metric in param_grid['metric']:
                for eps in param_grid['cluster_selection_epsilon']:
                    
                    print(f"Testing: min_cluster_size={min_cs}, min_samples={min_s}, "
                        f"metric={metric}, epsilon={eps}", end=" ")
                    
                    # Fit HDBSCAN
                    clusterer = hdbscan.HDBSCAN(
                        min_cluster_size=min_cs,
                        min_samples=min_s,
                        metric=metric,
                        cluster_selection_epsilon=eps,
                        core_dist_n_jobs=-1,  # Use all CPU cores
                        gen_min_span_tree=True  # <-- required for relative_validity_
                    )
                    
                    labels = clusterer.fit_predict(X_tune)
                    
                    # Calculate metrics
                    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                    n_noise = list(labels).count(-1)
                    pct_noise = (n_noise / len(labels)) * 100
                    
                    # DBCV score (HDBSCAN's native validation metric)
                    dbcv_score = clusterer.relative_validity_
                    
                    # Traditional metrics (only if we have clusters)
                    if n_clusters > 1:
                        mask = labels != -1
                        if mask.sum() > 1:
                            db_score = davies_bouldin_score(X_tune[mask], labels[mask])
                            ch_score = calinski_harabasz_score(X_tune[mask], labels[mask])
                        else:
                            db_score = ch_score = np.nan
                    else:
                        db_score = ch_score = np.nan
                    
                    # Store results
                    result = {
                        'min_cluster_size': min_cs,
                        'min_samples': min_s,
                        'metric': metric,
                        'cluster_selection_epsilon': eps,
                        'n_clusters': n_clusters,
                        'n_noise': n_noise,
                        'pct_noise': pct_noise,
                        'dbcv': dbcv_score,
                        'davies_bouldin': db_score,
                        'calinski_harabasz': ch_score,
                    }
                    
                    tuning_results.append(result)
                    
                    print(f"-> Clusters: {n_clusters}, Noise: {pct_noise:.1f}%, DBCV: {dbcv_score:.4f}")

    # Convert to DataFrame for analysis
    import pandas as pd
    results_df = pd.DataFrame(tuning_results)

    print("\n" + "="*80)
    print("TUNING RESULTS SUMMARY")
    print("="*80)

    # Sort by DBCV (higher is better)
    results_df_sorted = results_df.sort_values('dbcv', ascending=False)

    print("\nTop 10 configurations by DBCV score:")
    print(results_df_sorted[['min_cluster_size', 'min_samples', 'metric', 
                            'cluster_selection_epsilon', 'n_clusters', 
                            'pct_noise', 'dbcv']].head(10).to_string(index=False))

    # Find balanced configurations
    # We want: high DBCV, reasonable noise % (5-20%), and 2-4 clusters
    balanced = results_df[
        (results_df['pct_noise'] >= 5) & 
        (results_df['pct_noise'] <= 20) &
        (results_df['n_clusters'] >= 2) &
        (results_df['n_clusters'] <= 4)
    ].sort_values('dbcv', ascending=False)

    print("\n" + "-"*80)
    print("Balanced configurations (5-20% noise, 2-4 clusters):")
    if len(balanced) > 0:
        print(balanced[['min_cluster_size', 'min_samples', 'metric', 
                        'cluster_selection_epsilon', 'n_clusters', 
                        'pct_noise', 'dbcv']].head(5).to_string(index=False))
    else:
        print("No configurations meet the balanced criteria")

    # Select best configuration
    if len(balanced) > 0:
        best_config = balanced.iloc[0]
    else:
        best_config = results_df_sorted.iloc[0]

    print("\n" + "="*80)
    print("SELECTED BEST CONFIGURATION:")
    print("="*80)
    print(f"min_cluster_size: {best_config['min_cluster_size']}")
    print(f"min_samples: {best_config['min_samples']}")
    print(f"metric: {best_config['metric']}")
    print(f"cluster_selection_epsilon: {best_config['cluster_selection_epsilon']}")
    print(f"\nExpected Performance:")
    print(f"  - Number of clusters: {best_config['n_clusters']}")
    print(f"  - Noise points: {best_config['pct_noise']:.2f}%")
    print(f"  - DBCV score: {best_config['dbcv']:.4f}")
else:
    min_cluster_size= 1000
    min_samples= 15
    metric= "euclidean"
    cluster_selection_epsilon= 0.2

### 7.4. Train Final Model with Best Hyperparameters

Now that we have everything ready, it's time to train the final model with everything we learned and the hyperparameters we just tuned. Likewe saw earlier, HDBSCAN is the model that works best for our case. We selected HDBSCAN (Hierarchical DBSCAN) as the final model because it overcomes the limitations of K-Means and standard DBSCAN.It does not require specifying $k$ beforehand.It is robust to variable densities (unlike DBSCAN).

HDBSCAN achieved the highest validity score. It successfully separated the dense "Normal" traffic from the sparse "Anomalous" points without forcing ambiguous points into a cluster. This reduces false positives compared to K-Means.

In [ ]:
if PARAMETER_TUNING:
    final_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=int(best_config['min_cluster_size']),
        min_samples=int(best_config['min_samples']),
        metric=best_config['metric'],
        cluster_selection_epsilon = float(best_config['cluster_selection_epsilon']),
        core_dist_n_jobs=-1,
        gen_min_span_tree=True,
        prediction_data=True  # Enable soft clustering for new data
    )
else:
    final_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric=metric,
        cluster_selection_epsilon = cluster_selection_epsilon,
        core_dist_n_jobs=-1,
        gen_min_span_tree=True,
        prediction_data=True  # Enable soft clustering for new data
    )
print(f"\nFitting on full dataset ({X_scaled.shape[0]} samples)...")
labels = final_clusterer.fit_predict(X_scaled)

# Get outlier scores (higher = more anomalous)
outlier_scores = final_clusterer.outlier_scores_

# Get cluster probabilities
probabilities = final_clusterer.probabilities_

print("\nFinal Model Statistics:")
print(f"  - Total samples: {len(labels)}")
print(f"  - Clusters found: {len(set(labels)) - (1 if -1 in labels else 0)}")
print(f"  - Noise points: {list(labels).count(-1)} ({(list(labels).count(-1)/len(labels))*100:.2f}%)")
print(f"  - DBCV score: {final_clusterer.relative_validity_:.4f}")

# Cluster size distribution
unique_labels, counts = np.unique(labels, return_counts=True)
print("\nCluster distribution:")
for label, count in zip(unique_labels, counts):
    pct = (count / len(labels)) * 100
    cluster_name = "Noise" if label == -1 else f"Cluster {label}"
    print(f"  {cluster_name}: {count:,} samples ({pct:.2f}%)")

### 7.5. Yellowbrick Visualizations

Now to check the results of our model, we use yellowbrick to create some visualizations.

Silhouette Analysis: This metric measures how similar an object is to its own cluster (cohesion) compared to other clusters (separation). It provides a graphical representation of how well-separated and dense the resulting clusters are.

Intercluster Distance Map: Since the data is high-dimensional (33+ features), it is difficult to understand the relationship between clusters. This visualization embeds the cluster centers in 2D space to show the relative distance between groups. If the "Normal" cluster and "Anomaly" clusters overlap significantly in this map, the model might be weak.

In [ ]:
try:
    from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer, InterclusterDistance
    has_yellowbrick = True
except ImportError:
    print("Warning: Yellowbrick not installed. Skipping specialized visualizations.")
    print("Install with: pip install yellowbrick")
    has_yellowbrick = False

if has_yellowbrick:
    # Use PCA-reduced data for visualizations (computational efficiency)
    pca_viz = PCA(n_components=0.95, random_state=SEED)
    X_viz = pca_viz.fit_transform(X_scaled)
    
    print(f"\nUsing PCA-reduced data: {X_viz.shape[1]} components")
    
    # 1. Silhouette Analysis for labeled clusters only
    print("\n1. Generating Silhouette Analysis...")
    mask_labeled = labels != -1
    
    if mask_labeled.sum() > 0 and len(set(labels[mask_labeled])) > 1:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Create a temporary KMeans model with the number of found clusters for visualization
        from sklearn.cluster import KMeans
        n_viz_clusters = len(set(labels[mask_labeled]))
        kmeans_viz = KMeans(n_clusters=n_viz_clusters, random_state=SEED)
        
        visualizer = SilhouetteVisualizer(kmeans_viz, colors='yellowbrick', ax=ax)
        visualizer.fit(X_viz[mask_labeled])
        visualizer.finalize()
        plt.title('Silhouette Analysis (Labeled Clusters Only)')
        plt.tight_layout()
        plt.show()
    
    # 2. Intercluster Distance Map
    print("\n2. Generating Intercluster Distance Map...")
    if len(set(labels[mask_labeled])) > 1:
        fig, ax = plt.subplots(figsize=(8, 8))
        
        kmeans_viz2 = KMeans(n_clusters=n_viz_clusters, random_state=SEED)
        visualizer = InterclusterDistance(kmeans_viz2, ax=ax)
        visualizer.fit(X_viz[mask_labeled])
        visualizer.finalize()
        plt.title('Intercluster Distance Map')
        plt.tight_layout()
        plt.show()

#### 7.5.1. Cluster assigments in 2D

We project the high-dimensional dataset down to 2 dimensions using Principal Component Analysis (PCA). It then plots these points on a scatter plot, coloring them according to the labels assigned by the final HDBSCAN model (e.g., Cluster 0, Cluster 1, Noise). Humans cannot visualize that many dimensions, projecting it in 2D helps visualize it. The PCA step calculates how much information is preserved during compression.

Compressing the data from ~34 dimensions down to just 2 resulted in almost no loss of information. Therefore, the 2D plot is a highly accurate representation of the network traffic structure, and the separation seen visually is "real," not an artifact of the projection.

In [ ]:
# Prepare 2D projection for visualization
pca_2d = PCA(n_components=2, random_state=SEED)
X_2d = pca_2d.fit_transform(X_scaled)

print(f"\n2D PCA explains {pca_2d.explained_variance_ratio_.sum():.2%} of variance")

# Create comprehensive visualization
fig = plt.figure(figsize=(20, 12))

# 1. Cluster assignments in 2D
ax1 = plt.subplot(2, 3, 1)
scatter = ax1.scatter(X_2d[:, 0], X_2d[:, 1], c=labels, cmap='tab10', 
                      alpha=0.6, s=10, edgecolors='none')
noise_mask = labels == -1
ax1.scatter(X_2d[noise_mask, 0], X_2d[noise_mask, 1], c='black', 
           marker='x', s=30, alpha=0.5, label='Anomalies')
ax1.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
ax1.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
ax1.set_title('HDBSCAN Cluster Assignments')
ax1.legend()
ax1.grid(alpha=0.3)

#### 7.5.2. Outlier scores heatmap

A scatter plot is created using the 2D PCA projection of the data (just like in 7.5.1). However, instead of coloring by cluster label, the points are colored by their GLOSH (Global-Local Outlier Score from Hierarchies) score using a "Yellow-Orange-Red" color map.

While the previous plot showed where the clusters are, this plot shows how anomalous specific areas are. It highlights the gradient of "normalcy." Anomalies usually do not live in the center of a density blob; they live on the edges. This visualization helps confirm if the outlier scores increase as points move further away from the cluster centers.

In [ ]:
ax2 = plt.subplot(2, 3, 2)
scatter = ax2.scatter(X_2d[:, 0], X_2d[:, 1], c=outlier_scores, 
                     cmap='YlOrRd', alpha=0.7, s=10, edgecolors='none')
plt.colorbar(scatter, ax=ax2, label='Outlier Score')
ax2.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
ax2.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
ax2.set_title('HDBSCAN Outlier Scores (Higher = More Anomalous)')
ax2.grid(alpha=0.3)

#### 7.5.3. Cluster membership probabilities

Another scatter plot on the 2D PCA projection. This time, points are colored based on the probability (confidence) that they belong to their assigned cluster.

- Darker colors: Low confidence (the point is on the "fence" or is noise).

- Brighter/Yellow colors: High confidence (the point is definitely part of the cluster).

This plot visualizes the model's uncertainty. If a cluster is entirely dark/low probability, it might be a weak cluster that shouldn't be trusted. Strong clusters have bright, dense cores.

In [ ]:
ax3 = plt.subplot(2, 3, 3)
scatter = ax3.scatter(X_2d[:, 0], X_2d[:, 1], c=probabilities, 
                     cmap='viridis', alpha=0.7, s=10, edgecolors='none')
plt.colorbar(scatter, ax=ax3, label='Membership Probability')
ax3.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.1%})')
ax3.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.1%})')
ax3.set_title('Cluster Membership Confidence')
ax3.grid(alpha=0.3)

#### 7.5.4. Outlier score distribution

A histogram shows the frequency of the outlier scores across the entire dataset. Vertical dashed lines are drawn at the 95th and 99th percentiles. Most data should be at the bottom and only the outliers we are interested in will be at the top.

In [ ]:
ax4 = plt.subplot(2, 3, 4)
ax4.hist(outlier_scores, bins=50, color='coral', edgecolor='black', alpha=0.8)
ax4.axvline(np.percentile(outlier_scores, 95), color='red', linestyle='--', 
           linewidth=2, label='95th percentile')
ax4.axvline(np.percentile(outlier_scores, 99), color='darkred', linestyle='--', 
           linewidth=2, label='99th percentile')
ax4.set_xlabel('Outlier Score')
ax4.set_ylabel('Frequency')
ax4.set_title('Distribution of Outlier Scores')
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

#### 7.5.5. Cluster size comparison

A bar chart counts the number of samples in each cluster. The "Noise" cluster (label -1) is explicitly colored black to stand out. Percentage labels are added to show the relative size of each group. In anomaly detection, "Normal" traffic should be the vast majority. If the "Anomalies" bar is the tallest one, the model is likely broken (too sensitive). It tells the security team how many alerts they might face. If the "Noise" bar represents 20% of the data, that's too many alerts. If it's 1-5%, it's manageable.

In [ ]:
ax5 = plt.subplot(2, 3, 5)
cluster_labels = [f"Cluster {l}" if l != -1 else "Anomalies" for l in unique_labels]
colors_bar = ['black' if l == -1 else f'C{l}' for l in unique_labels]
bars = ax5.bar(cluster_labels, counts, color=colors_bar, edgecolor='black', alpha=0.8)
ax5.set_ylabel('Number of Samples')
ax5.set_title('Cluster Size Distribution')
ax5.tick_params(axis='x', rotation=45)
ax5.grid(axis='y', alpha=0.3)

# Add percentage labels on bars
for bar, count in zip(bars, counts):
    height = bar.get_height()
    pct = (count / len(labels)) * 100
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)

#### 7.5.6. Condensed tree (HDBSCAN hierarchy)

This plots the condensed cluster tree, a specialized dendrogram used by HDBSCAN. It visualizes how the algorithm decided to keep certain clusters and discard others as it varied the density threshold. The branches that persist the longest (are the longest vertically) represent the most stable, real clusters.

In [ ]:
ax6 = plt.subplot(2, 3, 6)
final_clusterer.condensed_tree_.plot(select_clusters=True, 
                                     selection_palette=plt.cm.tab10.colors,
                                     axis=ax6)
ax6.set_title('HDBSCAN Condensed Tree')

plt.tight_layout()
plt.show()

### 7.6. Anomaly Detection Validation

Since we don't have labels to validate our data, we need to use different methods. This section calculates the difference between the statistical properties of the "Normal" cluster and the "Anomaly" (Noise) group. It iterates through the features and compares the mean values of the normal traffic against the mean values of the detected anomalies. It prints out the features with the largest deviations ($\Delta$).

By comparing Normal: {val} vs Anomaly: {val}, the notebook translates abstract math into security insights (e.g., "Anomalies have 50x more bytes transferred than normal traffic"). The script identifies specific features that drive the anomaly detection. The final output identifies the exact number of "severe anomalies" (points classified as -1 by HDBSCAN with high outlier scores) and flags them as potential data exfiltration events that require investigation.

In [ ]:
# Define anomaly threshold (points with high outlier scores)
anomaly_threshold_95 = np.percentile(outlier_scores, 95)
anomaly_threshold_99 = np.percentile(outlier_scores, 99)

print(f"\nOutlier Score Thresholds:")
print(f"  - 95th percentile: {anomaly_threshold_95:.4f}")
print(f"  - 99th percentile: {anomaly_threshold_99:.4f}")

# Identify different levels of anomalies
severe_anomalies = outlier_scores >= anomaly_threshold_99
moderate_anomalies = (outlier_scores >= anomaly_threshold_95) & (outlier_scores < anomaly_threshold_99)
normal = outlier_scores < anomaly_threshold_95

print(f"\nAnomaly Classification:")
print(f"  - Severe anomalies (>99th): {severe_anomalies.sum():,} ({(severe_anomalies.sum()/len(labels))*100:.2f}%)")
print(f"  - Moderate anomalies (95-99th): {moderate_anomalies.sum():,} ({(moderate_anomalies.sum()/len(labels))*100:.2f}%)")
print(f"  - Normal (<95th): {normal.sum():,} ({(normal.sum()/len(labels))*100:.2f}%)")

# Compare feature distributions: Normal vs Anomalies
print("\n" + "-"*80)
print("FEATURE ANALYSIS: Normal vs Severe Anomalies")
print("-"*80)

# Get feature names
feature_names = [col for col in df_eda.columns if col in numeric_features]

# Calculate mean feature values for each group
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_names)
normal_features = X_scaled_df[normal].mean()
anomaly_features = X_scaled_df[severe_anomalies].mean()

# Find features with largest differences
feature_diff = np.abs(anomaly_features - normal_features)
top_discriminative = feature_diff.nlargest(10)

print("\nTop 10 features distinguishing anomalies from normal traffic:")
print("(Larger values = more discriminative)")
print("-"*60)
for feat, diff in top_discriminative.items():
    normal_val = normal_features[feat]
    anomaly_val = anomaly_features[feat]
    print(f"{feat:<40} Δ={diff:>6.3f}")
    print(f"  Normal: {normal_val:>7.3f}  |  Anomaly: {anomaly_val:>7.3f}")

print("\n" + "="*80)
print("MODEL VALIDATION COMPLETE")
print("="*80)
print(f"\nFinal model identifies {severe_anomalies.sum():,} severe anomalies")
print(f"These represent potential data exfiltration events and should be investigated")


### 7.7. Model persistence

We save the model we created, the preprocesing artifacts and the metada. This allows versioning for the models and documents the results. To detect exfiltration in real-time, you must apply the exact same scaling rules to new incoming traffic before passing it to the model. Saving the scaler ensures the new data is mathematically compatible with the trained model.

In [ ]:
import joblib
import json
from datetime import datetime

# 1. Definir y crear el directorio de modelos
models_dir = Path.cwd() / "models"
models_dir.mkdir(exist_ok=True)

# 2. Generar un timestamp para el versionado (Clave para Nivel 3)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 3. Definir rutas completas
model_path = models_dir / f"hdbscan_exfiltration_{timestamp}.pkl"
scaler_path = models_dir / f"robust_scaler_{timestamp}.pkl"
metadata_path = models_dir / f"model_metadata_{timestamp}.json"

# Guardar el modelo entrenado
joblib.dump(final_clusterer, model_path)

# Guardar el escalador (imprescindible para el despliegue/MLOps)
joblib.dump(scaler, scaler_path)

# Guardar metadatos (Hiperparámetros y métricas de rendimiento)
# Esto asegura el "seguimiento de experimentos" 
metadata = {
    'timestamp': timestamp,
    'labels': labels,
    'n_samples': len(labels),
    'n_features': X_scaled.shape[1],
    'hyperparameters': {
        'min_cluster_size': min_cluster_size,
        'min_samples': min_samples,
        'metric': metric,
        'epsilon': cluster_selection_epsilon
    },
    'metrics': {
        'clusters_found': len(set(labels)) - (1 if -1 in labels else 0),
        'pct_noise': (list(labels).count(-1) / len(labels)) * 100,
        'dbcv_score': float(final_clusterer.relative_validity_)
    }
}

with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=4)

def upload_to_s3(local_file, bucket, s3_prefix):
    if not USE_S3:
        return
    
    try:
        s3 = boto3.client('s3')
        s3_path = f"{s3_prefix}{local_file.name}"
        s3.upload_file(str(local_file), bucket, s3_path)
        print(f"Uploaded to S3: {s3_path}")
    except Exception as e:
        print(f"Error uploading {local_file.name} to S3: {e}")

# Después de joblib.dump y json.dump en la celda 7.7:
if USE_S3:
    upload_to_s3(model_path, S3_BUCKET_NAME, S3_MODELS_PREFIX)
    upload_to_s3(scaler_path, S3_BUCKET_NAME, S3_MODELS_PREFIX)
    upload_to_s3(metadata_path, S3_BUCKET_NAME, S3_MODELS_PREFIX)

### 8. Model Interpretation

Unsupervised models, like HDBSCAN, often function as "black boxes"—they group data effectively but do not explicitly state why a specific flow was classified as an anomaly. To satisfy the requirement of Extracting Knowledge, we employ a Surrogate Model Strategy: we train a supervised classifier (Random Forest) to predict the clusters found by our unsupervised model. This allows us to use mature explainability tools like Feature Importance and SHAP to interpret the underlying behavioral patterns of the detected attacks. Thanks to all of this we can further understand and improve our model.

#### 8.1. Global Feature Importance

We calculate the Global Feature Importance (using Gini Impurity or Permutation Importance from the surrogate model) to answer a fundamental question: "Which network characteristics drive the separation between Normal traffic and Exfiltration?" This technique is essential for validating the model's logic. If the top features were irrelevant (e.g., random ports), the model would be flawed.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

rf_explainer = RandomForestClassifier(
    n_estimators=100, 
    random_state=SEED, 
    max_depth=7, 
    n_jobs=-1,
    class_weight='balanced'
)

rf_explainer.fit(X_scaled, labels)

surrogate_preds = rf_explainer.predict(X_scaled)
fidelity_score = rf_explainer.score(X_scaled, labels)

print(f"\nSurrogate Model Fidelity: {fidelity_score:.4f}")
print("If fidelity is > 0.90, explanations are reliable representation of the clusters.")
print("\nClassification Report (Cluster Prediction):")
print(classification_report(labels, surrogate_preds))

importances = rf_explainer.feature_importances_
df_importance = pd.DataFrame({
    'Feature': numeric_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=df_importance.head(20), palette='viridis')
plt.title('Global Feature Importance\n(Variables driving the Cluster Separation)')
plt.xlabel('Gini Importance')
plt.ylabel('Feature')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 5 Drivers of Cluster Separation:")
for i, row in df_importance.head(5).iterrows():
    print(f"- {row['Feature']}: {row['Importance']:.4f}")

#### 8.2. Local Explainability (SHAP Analysis)

Global importance gives us an average view, but we also need to understand individual incidents. We use SHAP (SHapley Additive exPlanations) values to generate "Local Explanations". The SHAP Summary Plot (Beeswarm) visualizes not just which features are important, but how they influence the decision (e.g., Does a high value indicate an attack or normal behavior?).

- Red dots (High values) of fwd_pkts_payload.tot push the prediction strongly towards the Anomaly/Attack Class.

- Blue dots (Low values) of flow_duration push the prediction towards the Normal Class.

This allows us to explain specific false positives. For instance, if a legitimate backup process is flagged as an attack, SHAP would show us it was due to its high payload, allowing the analyst to whitelist that specific behavior. This improves our knowledge on the model.

In [ ]:
try:
    import shap
    print("Calculating SHAP values...")

    explainer = shap.TreeExplainer(rf_explainer)

    # Sampling for performance
    if len(X_scaled) > 1000:
        rng = np.random.default_rng(SEED)
        shap_sample_idx = rng.choice(len(X_scaled), 500, replace=False)
        X_shap_sample = X_scaled[shap_sample_idx]
        y_shap_sample = labels[shap_sample_idx]
    else:
        X_shap_sample = X_scaled
        y_shap_sample = labels

    # Convert to DF so SHAP can align feature names
    X_shap_df = pd.DataFrame(X_shap_sample, columns=numeric_features)

    # Multi-class shap values: list of arrays, one per class
    shap_values = explainer.shap_values(X_shap_df)

    classes = rf_explainer.classes_
    print(f"Classes found: {classes}")

    # Prefer noise (-1) if present, else smallest class
    if -1 in classes:
        target_class_val = -1
    else:
        class_counts = pd.Series(y_shap_sample).value_counts()
        target_class_val = class_counts.idxmin()

    target_idx = np.where(classes == target_class_val)[0][0]

    print(f"Visualizing SHAP Summary for Class {target_class_val}")

    # Select and plot
    target_shap = shap_values[target_idx]

    if target_shap.shape[1] != X_shap_df.shape[1]:
        raise ValueError(
            f"SHAP mismatch: values({target_shap.shape}) vs data({X_shap_df.shape})"
        )

    shap.summary_plot(
        target_shap,
        X_shap_df,
        feature_names=numeric_features,
        plot_type="dot"
    )

except ImportError:
    print("SHAP library not installed. Skipping visualization.")


#### 8.3. Cluster Profiling (Knowledge Extraction)

Finally, we perform Cluster Profiling. We compute the median values of the original (non-scaled) features for each cluster to give them a semantic meaning.

Cluster 0 (Normal): Characterized by low duration (< 1s) and low payload (< 1KB). Represents standard web browsing or API calls.

Cluster 1 (Potential Exfiltration): Characterized by extremely high payload (> 100MB) and long duration.

Noise (-1): Highly irregular points that do not fit standard patterns, often indicative of scanning or failed attempts

This profiling translates mathematical clusters into actionable threat intelligence, enabling the security team to write specific firewall rules based on these thresholds. This improves our knowledge on the model and could help us improve it.

In [ ]:
# 1. Recover Original Data (Unscaled)
df_profiling = LAZYFRAME.select(numeric_features).collect().to_pandas()

# 2. Add Cluster Labels
df_profiling['Cluster'] = labels

# 3. Calculate Statistics per Cluster
# We use Median because Mean is skewed by outliers
cluster_profile = df_profiling.groupby('Cluster').median()

# 4. Add Count of samples per cluster to see size
cluster_counts = df_profiling['Cluster'].value_counts().sort_index()
cluster_profile['Sample_Count'] = cluster_counts

# 5. Select Key Features to Display
# Showing 50 columns is too much. We show the top features from Section 8.1
top_features = df_importance['Feature'].head(7).tolist() + ['Sample_Count']

print("Cluster Profiles (Median Values - Original Scale):")
print("-" * 60)
display(cluster_profile[top_features].T.style.background_gradient(cmap='Reds', axis=1))

print("\nAutomatic Interpretation:")
for cluster_id in cluster_profile.index:
    count = cluster_profile.loc[cluster_id, 'Sample_Count']
    duration = cluster_profile.loc[cluster_id, 'flow_duration'] if 'flow_duration' in cluster_profile.columns else 0
    bytes_tot = cluster_profile.loc[cluster_id, 'fwd_pkts_payload.tot'] if 'fwd_pkts_payload.tot' in cluster_profile.columns else 0
    
    label_name = "Unknown"
    if cluster_id == -1:
        label_name = "NOISE / ANOMALY (Potential Attack or Scan)"
    elif bytes_tot > 1000000: # Example threshold 1MB
        label_name = "HIGH VOLUME TRANSFER (Exfiltration Candidate)"
    elif duration < 1:
        label_name = "SHORT FLOWS (Interactive/Background)"
    
    print(f"Cluster {cluster_id} ({count} samples): {label_name}")
    print(f"  -> Median Duration: {duration:.2f}s | Median Payload: {bytes_tot:.0f} bytes")